# FastAPI Multi-Agent System Testing

This notebook demonstrates how to test and interact with the FastAPI endpoints.

## Prerequisites

Make sure the FastAPI server is running:
```bash
python api.py
# OR
uvicorn api:app --host 0.0.0.0 --port 8000 --reload
```

## Setup and Imports

In [2]:
import requests
import json
import asyncio
import websockets
from pathlib import Path
import time
from IPython.display import display, Markdown, HTML

# API Base URL
BASE_URL = "http://localhost:8000"

print("✅ Imports successful")
print(f"🌐 API Base URL: {BASE_URL}")

✅ Imports successful
🌐 API Base URL: http://localhost:8000


## 1. Health Check

Test if the API is running and all agents are initialized.

In [3]:
def test_health_check():
    """Test the health check endpoint."""
    try:
        response = requests.get(f"{BASE_URL}/health")
        response.raise_for_status()
        data = response.json()
        
        print("🏥 Health Check Response:")
        print(f"  Status: {data['status']}")
        print(f"  Initialized: {data['initialized']}")
        print("\n  Agent Status:")
        for agent, status in data['agents'].items():
            emoji = "✅" if status else "❌"
            print(f"    {emoji} {agent}: {status}")
        
        return data
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

health_data = test_health_check()

🏥 Health Check Response:
  Status: healthy
  Initialized: True

  Agent Status:
    ✅ crypto: True
    ✅ rag: True
    ✅ stock: True
    ✅ search: True
    ✅ finance_tracker: True


## 2. Root Endpoint

Get API information and available endpoints.

In [4]:
def test_root_endpoint():
    """Test the root endpoint."""
    try:
        response = requests.get(f"{BASE_URL}/")
        response.raise_for_status()
        data = response.json()
        
        print("📋 API Information:")
        print(json.dumps(data, indent=2))
        
        return data
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

api_info = test_root_endpoint()

📋 API Information:
{
  "message": "Multi-Agent Assistant API",
  "version": "1.0.0",
  "docs": "/docs",
  "health": "/health"
}


## 3. Streaming Chat with Server-Sent Events (SSE)

Test the streaming chat endpoint that returns real-time updates.

In [5]:
def test_streaming_chat(message, history=None):
    """Test streaming chat with SSE."""
    if history is None:
        history = []
    
    payload = {
        "message": message,
        "history": history
    }
    
    print(f"💬 Query: {message}")
    print("\n📡 Streaming Response:")
    print("=" * 80)
    
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/chat/stream",
            json=payload,
            headers={"Accept": "text/event-stream"},
            stream=True
        )
        response.raise_for_status()
        
        final_answer = ""
        
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8')
                if line_str.startswith('data: '):
                    data_str = line_str[6:]  # Remove 'data: ' prefix
                    try:
                        event = json.loads(data_str)
                        event_type = event.get('type', 'unknown')
                        
                        if event_type == 'thinking':
                            print(f"\n💭 Step {event.get('step', '?')}: Reasoning")
                            print(f"   Thought: {event.get('thought', 'N/A')}")
                            print(f"   Action: {event.get('action', 'N/A').upper()}")
                            print(f"   Justification: {event.get('justification', 'N/A')}")
                        
                        elif event_type == 'action':
                            agent = event.get('agent', 'unknown')
                            print(f"\n🔧 Calling {agent.title()} Agent...")
                        
                        elif event_type == 'observation':
                            agent = event.get('agent', 'unknown')
                            summary = event.get('summary', '')
                            print(f"\n📊 {agent.title()} Agent Results:")
                            print(f"   {summary[:200]}..." if len(summary) > 200 else f"   {summary}")
                        
                        elif event_type == 'final_start':
                            print("\n✨ Final Answer:")
                            print("-" * 80)
                        
                        elif event_type == 'final_token':
                            final_answer = event.get('accumulated', '')
                        
                        elif event_type == 'final_complete':
                            print(final_answer)
                            print("-" * 80)
                        
                        elif event_type == 'error':
                            print(f"\n❌ Error: {event.get('error', 'Unknown error')}")
                    
                    except json.JSONDecodeError:
                        continue
        
        print("\n" + "=" * 80)
        return final_answer
    
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

# Test with a simple crypto query
result = test_streaming_chat("What is the current price of Bitcoin?")

💬 Query: What is the current price of Bitcoin?

📡 Streaming Response:

💭 Step 1: Reasoning
   Thought: The user is asking for the current price of Bitcoin. The `CALL_CRYPTO` agent is designed to get cryptocurrency market data and prices. This is the most direct and appropriate tool to answer the query.
   Action: CRYPTO
   Justification: This action will directly retrieve the current price of Bitcoin, which is precisely what the user is asking for.

🔧 Calling Crypto Agent...

📊 Crypto Agent Results:
   The current price of Bitcoin is $95,748.

💭 Step 2: Reasoning
   Thought: I have already called the `CALL_CRYPTO` agent, which provided the current price of Bitcoin as $95,748. This directly answers the user's query. I have sufficient information to provide a final answer.
   Action: FINISH
   Justification: The `CALL_CRYPTO` agent has successfully retrieved the current price of Bitcoin, which was the user's original query. No further information is needed.

✨ Final Answer:
-------------

### Test with Different Queries

In [6]:
# Test crypto query
test_streaming_chat("What are the top 5 cryptocurrencies by market cap?")

💬 Query: What are the top 5 cryptocurrencies by market cap?

📡 Streaming Response:

💭 Step 1: Reasoning
   Thought: The user is asking for a list of the top 5 cryptocurrencies based on their market capitalization. This is a specific query about cryptocurrency market data. I have an action called `CALL_CRYPTO` which is designed to get cryptocurrency market data, prices, and trends. This is the most direct and specialized tool for the job.
   Action: CRYPTO
   Justification: The `CALL_CRYPTO` action is the most appropriate tool to get real-time market capitalization data for cryptocurrencies, which is exactly what is needed to answer the user's query.

🔧 Calling Crypto Agent...

📊 Crypto Agent Results:
   The top 5 cryptocurrencies by market cap are:

1. Bitcoin (BTC) - Market Cap: $2,025,171,098,463 - Price: $101,536
2. Ethereum (ETH) - Market Cap: $411,695,018,306 - Price: $3,410.68
3. Tether (USDT) ...

💭 Step 2: Reasoning
   Thought: The CRYPTO agent has provided a list of the top 5 

'Based on current market data, here are the top 5 cryptocurrencies ranked by market capitalization:\n\n**1. Bitcoin (BTC)**\n*   **Market Cap:** $2,025,171,098,463\n*   **Price:** $101,536.00\n\n**2. Ethereum (ETH)**\n*   **Market Cap:** $411,695,018,306\n*   **Price:** $3,410.68\n\n**3. Tether (USDT)**\n*   **Market Cap:** $184,005,021,987\n*   **Price:** $0.999813\n\n**4. XRP (XRP)**\n*   **Market Cap:** $143,300,134,340*   **Price:** $2.38\n\n**5. BNB (BNB)**\n*   **Market Cap:** $131,225,068,810\n*   **Price:** $952.81\n\n**Key Insight:**\n\nMarket capitalization is a key metric used to determine the relative size and value of a cryptocurrency. It is calculated by multiplying the current market price of a single coin by the total number of coins in circulation. As shown above, Bitcoin holds a dominant position with a market cap significantly larger than the others combined.\n\n*It is important to note that the cryptocurrency market is highly volatile, and these rankings, market cap

In [7]:
# Test stock query
test_streaming_chat("What is the current stock price of Apple (AAPL)?")

💬 Query: What is the current stock price of Apple (AAPL)?

📡 Streaming Response:

💭 Step 1: Reasoning
   Thought: The user is asking for the current stock price of Apple (AAPL). The `CALL_STOCK` action is the most appropriate tool for this query as it is designed to get stock market data. I will use this action to retrieve the price for the ticker "AAPL".
   Action: STOCK
   Justification: This action is specifically designed to retrieve stock market data, which is exactly what is needed to answer the user's query about Apple's stock price.

🔧 Calling Stock Agent...

📊 Stock Agent Results:
   The current stock price of Apple (AAPL) is $273.47.

💭 Step 2: Reasoning
   Thought: I have successfully retrieved the current stock price of Apple (AAPL) from the STOCK agent. This information directly answers the user's query. Therefore, I have sufficient information and can provide the final answer.
   Action: FINISH
   Justification: The STOCK agent has provided the exact information requested

'Based on the real-time data retrieved, the current stock price of Apple Inc. (AAPL) is **$273.47**.\n\nPlease be aware that this price reflects the market value at the time the query was made and is subject to constant fluctuation based on market activity.'

In [7]:
# Test web search query
test_streaming_chat("Search for the latest AI developments in 2025")

💬 Query: Search for the latest AI developments in 2025

📡 Streaming Response:

💭 Step 1: Reasoning
   Thought: The user is asking about future AI developments in 2025. This is a general knowledge and predictive query. I have no information gathered yet. The best tool to find information about current events, news, and future trends is the web search tool. The other tools (CRYPTO, STOCK, FINANCE_TRACKER, RAG) are not relevant to this topic. Therefore, I should use CALL_SEARCH to find the requested information.
   Action: SEARCH
   Justification: This action is the most appropriate for finding information on future technological trends and developments like those anticipated for AI in 2025.

🔧 Calling Search Agent...

📊 Search Agent Results:
   I couldn't find any results for "latest AI developments in 2025." Since 2025 is in the future, there are no "developments" yet. I can search for predictions or trends for AI in 2025 if you'd like.

💭 Step 2: Reasoning
   Thought: The initial searc

'I was unable to find information on the latest AI developments for 2025.\n\nMy attempt to answer your query involved a multi-step search process. Initially, I searched for "latest AI developments in 2025," but since 2025 is in the future, no "developments" have occurred yet. I then broadened my search to look for "AI predictions and trends for 2025" and subsequently the "future of AI."\n\nUnfortunately, all of these searches failed to return any results. This indicates a potential temporary issue with the search tool or that information specifically categorized for 2025 is not widely available at this moment.\n\nI recommend trying your query again later, as the issue with the search tool may be resolved.'

In [ ]:
# Test finance tracker - adding a transaction
test_streaming_chat("Add 10 shares of AAPL that I bought at $150")

In [8]:
# Test multi-agent query (requires multiple agents)
test_streaming_chat("What's Bitcoin's price and what's the latest news about cryptocurrency?")

💬 Query: What's Bitcoin's price and what's the latest news about cryptocurrency?

📡 Streaming Response:

💭 Step 1: Reasoning
   Thought: The user is asking for two pieces of information: the current price of Bitcoin and the latest news about cryptocurrency. I need to address both parts of the query. The `CALL_CRYPTO` action is the most direct way to get the price of Bitcoin. I will start with that.
   Action: CRYPTO
   Justification: This action is specifically designed to retrieve cryptocurrency market data and prices, which directly answers the first part of the user's query about Bitcoin's price.

🔧 Calling Crypto Agent...

📊 Crypto Agent Results:
   I can only fetch the price of Bitcoin at the moment. Bitcoin's current price is $101,568. I cannot get the latest news about cryptocurrency.

💭 Step 2: Reasoning
   Thought: I have successfully obtained the price of Bitcoin from the CRYPTO agent. However, the user's query also asks for the latest news about cryptocurrency, which the CRY

'The current price of Bitcoin is **$101,568**.\n\nFor the latest news and in-depth coverage of the cryptocurrency market, you can refer to these reputable sources:\n\n*   **CoinDesk:** [https://www.coindesk.com/](https://www.coindesk.com/)\n*   **Cointelegraph:** [https://cointelegraph.com/](https://cointelegraph.com/)\n*   **Crypto News:** [https://cryptonews.com/](https://cryptonews.com/)\n*   **Forbes Digital Assets:** [https://www.forbes.com/digital-assets/news/](https://www.forbes.com/digital-assets/news/)\n*   **Reuters Cryptocurrency:** [https://www.reuters.com/markets/cryptocurrency/](https://www.reuters.com/markets/cryptocurrency/)'

## 4. WebSocket Chat

Test the WebSocket endpoint for real-time bidirectional communication.

In [9]:
async def test_websocket_chat(message, history=None):
    """Test WebSocket chat endpoint."""
    if history is None:
        history = []
    
    ws_url = BASE_URL.replace('http://', 'ws://') + '/ws/v1/chat'
    
    print(f"💬 WebSocket Query: {message}")
    print("\n📡 WebSocket Response:")
    print("=" * 80)
    
    try:
        async with websockets.connect(ws_url) as websocket:
            # Send message
            await websocket.send(json.dumps({
                "message": message,
                "history": history
            }))
            
            final_answer = ""
            
            # Receive streaming updates
            while True:
                try:
                    response = await asyncio.wait_for(websocket.recv(), timeout=60.0)
                    event = json.loads(response)
                    event_type = event.get('type', 'unknown')
                    
                    if event_type == 'thinking':
                        print(f"\n💭 Step {event.get('step', '?')}: Reasoning")
                        print(f"   Thought: {event.get('thought', 'N/A')}")
                        print(f"   Action: {event.get('action', 'N/A').upper()}")
                    
                    elif event_type == 'action':
                        agent = event.get('agent', 'unknown')
                        print(f"\n🔧 Calling {agent.title()} Agent...")
                    
                    elif event_type == 'observation':
                        agent = event.get('agent', 'unknown')
                        summary = event.get('summary', '')
                        print(f"\n📊 {agent.title()} Agent Results")
                    
                    elif event_type == 'final_start':
                        print("\n✨ Final Answer:")
                        print("-" * 80)
                    
                    elif event_type == 'final_token':
                        final_answer = event.get('accumulated', '')
                    
                    elif event_type == 'final_complete':
                        print(final_answer)
                        print("-" * 80)
                        break
                    
                    elif event_type == 'error':
                        print(f"\n❌ Error: {event.get('error', 'Unknown error')}")
                        break
                
                except asyncio.TimeoutError:
                    print("\n⏱️ Timeout waiting for response")
                    break
            
            print("\n" + "=" * 80)
            return final_answer
    
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

# Test WebSocket
ws_result = await test_websocket_chat("What is Ethereum's current price?")

💬 WebSocket Query: What is Ethereum's current price?

📡 WebSocket Response:

💭 Step 1: Reasoning
   Thought: The user is asking for the current price of Ethereum. Ethereum is a cryptocurrency. The `CALL_CRYPTO` action is specifically designed to get cryptocurrency market data and prices. Therefore, this is the most direct and appropriate action to take.
   Action: CRYPTO

🔧 Calling Crypto Agent...

📊 Crypto Agent Results

💭 Step 2: Reasoning
   Thought: The user asked for the current price of Ethereum. The CRYPTO agent has already provided this information: "The current price of Ethereum is $3423.75." I have a direct and complete answer to the user's query. Therefore, I can now finish.
   Action: FINISH

✨ Final Answer:
--------------------------------------------------------------------------------
Based on the latest market data, the current price of Ethereum (ETH) is **$3,423.75**.

It is important to note that cryptocurrency prices are highly volatile and can change rapidly. This p

## 5. Document Upload

Test uploading a document to the RAG agent.

In [ ]:
def test_document_upload(file_path):
    """Test document upload endpoint."""
    try:
        if not Path(file_path).exists():
            print(f"❌ File not found: {file_path}")
            return None
        
        print(f"📤 Uploading: {file_path}")
        
        with open(file_path, 'rb') as f:
            files = {'file': (Path(file_path).name, f)}
            response = requests.post(
                f"{BASE_URL}/api/v1/documents/upload",
                files=files
            )
        
        response.raise_for_status()
        data = response.json()
        
        print("\n✅ Upload Response:")
        print(f"  Success: {data['success']}")
        print(f"  Message: {data['message']}")
        
        if data.get('details'):
            print("\n  Details:")
            for key, value in data['details'].items():
                print(f"    {key}: {value}")
        
        return data
    
    except requests.exceptions.HTTPError as e:
        print(f"\n❌ HTTP Error: {e}")
        print(f"   Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

# Example: Upload a test file (update path to your actual file)
test_document_upload("PATH_TO_YOUR_DOCUEMNT")

print("⚠️ To test document upload, uncomment the line above and provide a valid file path")

In [8]:
# Create a test text file and upload it
test_file_path = "test_document.txt"

with open(test_file_path, 'w') as f:
    f.write("""
    Test Document for RAG Agent
    
    This is a sample document for testing the RAG agent's document upload functionality.
    
    Key Information:
    - The multi-agent system uses Google Gemini 2.5 Pro
    - It implements a ReAct (Reasoning + Acting) pattern
    - Available agents: Crypto, Stock, RAG, Search, Finance Tracker
    - Each agent uses specialized MCP servers for their domain
    
    The system can handle co plex queries that require multiple agents working together.
    """)

print(f"✅ Created test file: {test_file_path}")

# Upload the test file
upload_result = test_document_upload(test_file_path)

✅ Created test file: test_document.txt
📤 Uploading: test_document.txt

✅ Upload Response:
  Success: True
  Message: Document uploaded successfully

  Details:
    filename: tmphdduvt25.txt
    file_type: .txt
    chunks_added: 1
    total_documents: 229


In [9]:
# Now query the uploaded document
if upload_result and upload_result.get('success'):
    test_streaming_chat("What applications of AI agents are described in my collection?")

💬 Query: What applications of AI agents are described in my collection?

📡 Streaming Response:

💭 Step 1: Reasoning
   Thought: The user is asking about applications of AI agents described in "my collection". This indicates that the information should be retrieved from the user's uploaded documents. The `CALL_RAG` tool is designed for this purpose.
   Action: RAG
   Justification: The query specifically asks for information from "my collection," which implies searching the user's uploaded documents. `CALL_RAG` is the appropriate tool to retrieve information from these documents.

🔧 Calling Rag Agent...

📊 Rag Agent Results:
   Based on the provided document excerpts, there is no information available regarding the current status of any project. The documents discuss applications of AI agents, challenges in their integration...

💭 Step 2: Reasoning
   Thought: The user asked for "What applications of AI agents are described in my collection?". I previously called `CALL_RAG`, and the obs

## 6. Conversation History

Test chat with conversation history.

In [6]:
# Start a conversation
history = []

# First message
print("\n" + "="*100)
print("MESSAGE 1")
print("="*100)
result1 = test_streaming_chat("What is Bitcoin?", history=history)
history.append({"role": "user", "content": "What is Bitcoin?"})
history.append({"role": "assistant", "content": result1})

time.sleep(2)

# Follow-up question (with context)
print("\n" + "="*100)
print("MESSAGE 2 (with context from previous message)")
print("="*100)
result2 = test_streaming_chat("What is its current price?", history=history)
history.append({"role": "user", "content": "What is its current price?"})
history.append({"role": "assistant", "content": result2})


MESSAGE 1
💬 Query: What is Bitcoin?

📡 Streaming Response:

💭 Step 1: Reasoning
   Thought: The user is asking for a definition of "Bitcoin". This is a general knowledge question. I do not have any information yet. The best tool for answering general knowledge questions is `CALL_SEARCH`. The other tools are more specialized: `CALL_CRYPTO` is for market data, not definitions; `CALL_STOCK` and `CALL_FINANCE_TRACKER` are for traditional finance; and `CALL_RAG` requires pre-existing documents. Therefore, I should use `CALL_SEARCH` to find out what Bitcoin is.
   Action: SEARCH
   Justification: The user's query "What is Bitcoin?" is a general knowledge question. The `CALL_SEARCH` action is the most appropriate tool to find a definition and explanation from the web.

🔧 Calling Search Agent...

📊 Search Agent Results:
   Bitcoin is a decentralized digital currency that allows for direct transactions between users without the need for an intermediary like a bank. It was invented in 2008 by a

## 7. Performance Testing

Test response times and measure performance.

In [26]:
import time

def measure_response_time(query):
    """Measure response time for a query."""
    payload = {"message": query, "history": []}
    
    start_time = time.time()
    first_token_time = None
    
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/chat/stream",
            json=payload,
            headers={"Accept": "text/event-stream"},
            stream=True
        )
        
        for i, line in enumerate(response.iter_lines()):
            if i == 0 and first_token_time is None:
                first_token_time = time.time() - start_time
            if line and b'final_complete' in line:
                break
        
        total_time = time.time() - start_time
        
        print(f"\n⏱️ Performance Metrics for: '{query}'")
        print(f"   Time to first token: {first_token_time:.2f}s")
        print(f"   Total response time: {total_time:.2f}s")
        
        return {"first_token": first_token_time, "total": total_time}
    
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Test with simple query
measure_response_time("What is Bitcoin price?")


⏱️ Performance Metrics for: 'What is Bitcoin price?'
   Time to first token: 14.65s
   Total response time: 36.27s


{'first_token': 14.646224975585938, 'total': 36.274803161621094}

## 8. Error Handling Tests

Test how the API handles various error conditions.

In [ ]:
# Test empty message
print("Testing empty message:")
test_streaming_chat("")

In [ ]:
# Test invalid file upload
print("Testing invalid file type:")

invalid_file_path = "/tmp/test.exe"
with open(invalid_file_path, 'w') as f:
    f.write("invalid content")

test_document_upload(invalid_file_path)

## 9. API Documentation

The FastAPI server provides automatic interactive documentation.

**Access the documentation at:**
- Swagger UI: http://localhost:8000/docs
- ReDoc: http://localhost:8000/redoc

You can test all endpoints directly from the browser using the Swagger UI!

In [25]:
from IPython.display import IFrame

# Display Swagger UI in notebook (if server is running)
print("📚 FastAPI Interactive Documentation")
print(f"\n🔗 Swagger UI: {BASE_URL}/docs")
print(f"🔗 ReDoc: {BASE_URL}/redoc")

# Uncomment to embed in notebook
# IFrame(f"{BASE_URL}/docs", width=1000, height=600)

📚 FastAPI Interactive Documentation

🔗 Swagger UI: http://localhost:8000/docs
🔗 ReDoc: http://localhost:8000/redoc
